In [15]:
import torch
import torch.nn as nn
import ast
from torch.utils.data import Dataset, DataLoader

import numpy as np

import numpy as np
import torch

class LandmarkSequenceDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        raw_string = self.data.iloc[idx]["Landmark_Sequence"]
        
        # Clean up the string and convert to a numpy array
        clean_str = raw_string.replace("[", "").replace("]", "").strip()
        sequence = np.fromstring(clean_str, sep=' ')
        
        # Check if sequence length matches the expected shape
        expected_length = 30 * 297
        if len(sequence) == expected_length:
            sequence_tensor = torch.tensor(sequence.reshape(30, 297), dtype=torch.float32)
        elif len(sequence) < expected_length:
            # If sequence is too short, pad it
            sequence_tensor = torch.tensor(np.pad(sequence, (0, expected_length - len(sequence))), dtype=torch.float32).reshape(30, 297)
        else:
            # If sequence is too long, truncate it
            sequence_tensor = torch.tensor(sequence[:expected_length], dtype=torch.float32).reshape(30, 297)

        # Get label and convert to tensor
        label = int(self.data.iloc[idx]["Label"])
        label_tensor = torch.tensor(label, dtype=torch.long)

        return sequence_tensor, label_tensor



# Model
class TransferBiLSTM(nn.Module):
    def __init__(self, input_size=132, hidden_size=64, num_layers=2, num_classes=2, freeze_layers=True):
        super(TransferBiLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                            batch_first=True, bidirectional=True)

        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size * 2, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, num_classes)
        )

        # Freeze LSTM if desired
        if freeze_layers:
            for param in self.lstm.parameters():
                param.requires_grad = False

    def forward(self, x):
        lstm_out, _ = self.lstm(x)  # x: (batch, seq_len, input_size)
        out = lstm_out[:, -1, :]  # last time step
        return self.classifier(out)



In [6]:
def train(model, dataloader, criterion, optimizer, device, epochs=10):
    model.to(device)
    model.train()

    for epoch in range(epochs):
        total_loss = 0
        correct = 0
        total = 0

        for sequences, labels in dataloader:
            sequences, labels = sequences.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(sequences)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss:.4f}, Accuracy: {100 * correct / total:.2f}%")


In [16]:
# Example setup
from sklearn.model_selection import train_test_split
import pandas as pd

final_df = pd.read_csv('/Users/suryanshpatel/Projects/Directed Readings/Technical/src/data/landmarks_sequences.csv')  # Load your DataFrame

# Let's say your DataFrame with sequences is called `final_df`
train_df, val_df = train_test_split(final_df, test_size=0.2, random_state=42)

train_dataset = LandmarkSequenceDataset(train_df)
val_dataset = LandmarkSequenceDataset(val_df)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# Instantiate model
model = TransferBiLSTM(freeze_layers=True)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train(model, train_loader, criterion, optimizer, device, epochs=15)


/var/folders/r2/_zqwhs3s7zd_mvl3kt6zzfsr0000gn/T/ipykernel_21007/4274306901.py:23: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  sequence = np.fromstring(clean_str, sep=' ')


RuntimeError: input.size(-1) must be equal to input_size. Expected 132, got 297

In [4]:
final_df.dtypes

Subject               int64
Trial                 int64
Start_Timestamp      object
End_Timestamp        object
Label                 int64
Landmark_Sequence    object
dtype: object

# Image model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, TensorDataset, random_split

# Convert numpy arrays to torch tensors
X = convert_to_3_channels(X).astype('float32') / 127.5 - 1.0  # MobileNetV2 preprocessing
y_tensor = torch.tensor(y, dtype=torch.long)
X_tensor = torch.tensor(X.transpose(0, 3, 1, 2), dtype=torch.float32)  # (N, C, H, W)

# Create dataset and split
dataset = TensorDataset(X_tensor, y_tensor)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Load MobileNetV2 pretrained
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.last_channel, 5)  # 5 classes
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_val_acc = 0.0
for epoch in range(5):
    model.train()
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            correct += (predicted == targets).sum().item()
            total += targets.size(0)
    val_acc = correct / total
    print(f"Epoch {epoch+1}, Validation Accuracy: {val_acc:.4f}")

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'mobilenetv2_model.pth')
